In [1]:
import numpy as np
import cv2
import os

In [2]:
def featureDetector(image):
    """
    Compute key points and feature descriptors using brisk method
    """
    # define descriptor
    descriptor = cv2.BRISK_create()
    # get keypoints and descriptors
    (kps, features) = descriptor.detectAndCompute(image, None)

    return (kps, features)

In [3]:
def bf(des1, des2, crossCheck=False):
    "Create a feature Matcher using BruteForce method"
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck).match(des1, des2)
    return bf

In [4]:
def perspective_transformation(source, img_to_stitch, H):
    """ align images together without cropping
    """
    
    # get image shapes
    h1, w1 = source.shape[:2]
    h2, w2 = img_to_stitch.shape[:2]
    
    # define the corner points for each image
    corners1 = np.array([[0, 0], [0, h1], [w1, h1], [w1, 0]])
    corners2 = np.array([[0, 0], [0, h2], [w2, h2], [w2, 0]])
    
    # trasnsform the obtained homography H to the corner points of the img_to_stitch
    transH = cv2.perspectiveTransform(np.float32([corners2]), H)[0]
    
    # join transformed homography with the corner points of source image
    newRect = np.concatenate((np.float32(corners1), transH), axis=0)
    
    # get the coordinates of the bounding rectangular of joined images
    # by taking the min & max
    [xmin, ymin] = np.int32(newRect.min(axis=0).ravel() - 0.5)
    [xmax, ymax] = np.int32(newRect.max(axis=0).ravel() + 0.5)
    
    # Compute the translation homography that will move (xmin, xmax) to (0, 0)
    th = np.array([
      [ 1, 0, -xmin],
      [ 0, 1, -ymin],
      [ 0, 0, 1]])
    
    # transform the homography to the new coordinates 
    newH = th.dot(H)
    
    # warp images together
    warp = cv2.warpPerspective(img_to_stitch, newH, (xmax-xmin, ymax-ymin))
    warp[-ymin:h1+ (-ymin), -xmin:w1+(-xmin)] = source
    
    return warp

In [11]:
def stitch_images(source, img_to_stitch):
    """ The function create panorama images by stitching two images together
    """
    gray_source = cv2.cvtColor(source, cv2.COLOR_BGR2GRAY)
    gray_img_to_stitch = cv2.cvtColor(img_to_stitch, cv2.COLOR_BGR2GRAY)
    # Compute key points and feature descriptors using the selected method
    kp1, des1 = featureDetector(gray_source)
    kp2, des2 = featureDetector(gray_img_to_stitch)

    # get feature matcher using bf
    
    matches = bf(des1, des2, crossCheck=True)
    # Sort the features in order of distance.
    # The points with small distance (more similarity) are ordered first in the vector
    matches = sorted(matches, key = lambda x:x.distance)
    print("Raw matches (Brute force):", len(matches))
        
    # convert the keypoints to numpy arrays
    kp1 = np.float32([kp.pt for kp in kp1])
    kp2 = np.float32([kp.pt for kp in kp2])
    
    # we need 4 matched points at least to match images
    # otherwise, there is nothing in common between images
    if len(matches) > 4:

        # construct the two sets of points
        pt1 = np.float32([kp1[m.queryIdx] for m in matches])
        pt2 = np.float32([kp2[m.trainIdx] for m in matches])

        # estimate the homography between the sets of points
        (H, mask) = cv2.findHomography(pt2, pt1, cv2.RANSAC, 4)
        
        warp = perspective_transformation(source, img_to_stitch, H)
        
        return warp

    else:
        return print('Images do not match')

### Stitch images

In [17]:
# list all images in the directory
images = ['images/'+ img for img in os.listdir('images/')]

Raw matches (Brute force): 5920
Raw matches (Brute force): 2629
Raw matches (Brute force): 3077
Raw matches (Brute force): 4118
Raw matches (Brute force): 6544
